In [ ]:
import os
import sys
import pandas as pd
from pynxtools_apm.utils.versioning import __version__
from pynxtools_apm.examples.get_sha256_of_directories import SEPARATOR
from pynxtools_apm.examples.oasisb_utils import APT_MIME_TYPES, CAMECA_ROOT_MIME_TYPES
from pynxtools_apm.examples.get_file_from_archive_formats import (
    get_file_from_zip,
    get_file_from_tar,
    get_file_from_rar,
    get_file_from_sevenzip,
)

# TODO parameterize
with open("root_directory.txt") as fp:
    root_directory = f"{fp.read().strip().replace('/', os.sep)}"
    src_directory = root_directory
    trg_directory = root_directory
print(src_directory)
print(trg_directory)

header = "file_path:archive_path;byte_size;unix_mtime;sha256sum"

In [ ]:
config: dict[str, str] = {
    "python_version": f"{sys.version.replace(' ', '_')}",
    "working_directory": f"{os.getcwd()}",
    "pynxtools_apm version": f"{__version__}",
    # "directory": f"src_directory,  # sys.argv[1],
}

ods = pd.read_excel(
    f"{src_directory}{os.sep}data{os.sep}aaa_legacy_data.ods",
    sheet_name="aaa_legacy_data",
    engine="odf",
)
cnt: dict[int, int] = {1: 0, 2: 0}
for row in ods.itertuples(index=True):
    if row.project_name not in ("mar_tomographic_hellman"):
        continue
    if row.parse in (1, 2):
        print(f"project{SEPARATOR}{row.project_name}{SEPARATOR}hashing...")
        sub_directory = f"{src_directory}{os.sep}data{os.sep}{row.project_name}"
        project_csv = (
            f"{src_directory}{os.sep}data{os.sep}{row.project_name}.sha256.results.csv"
        )
        with open(project_csv, "r") as fp:
            start = next(idx for idx, line in enumerate(fp) if header in line)
        df_hash = pd.read_csv(project_csv, sep=";", skiprows=start)
        del start, project_csv
        df_hash.columns = ["path", "size", "mtime", "sha256"]
        # print(df_hash)

        # build dictionary of hashes to replace original file names with short and clean unique ones
        file_to_hash: dict[str, str] = {}
        for line in df_hash.itertuples(index=True):
            if line.path.lower().endswith(
                tuple(APT_MIME_TYPES + CAMECA_ROOT_MIME_TYPES)
            ):
                file_to_hash[line.path] = line.sha256
        del df_hash

        # iv) unpack these files
        project_ods = f"{src_directory}{os.sep}data{os.sep}{row.project_name}.ods"
        df_workflow = pd.read_excel(
            project_ods,
            sheet_name=row.project_name,
            engine="odf",
        ).fillna("")
        del project_ods
        # print(df_workflow)

        # assume ods is the ground truth configuration with which to work
        n_entries = 0
        for row_idx in range(df_workflow.shape[0]):  # TODO ?? is the header skipped ?
            for col_idx in range(0, 6):  # TODO !!  df.shape[1])
                value = df_workflow.iat[row_idx, col_idx]
                if value == "":
                    continue
                else:
                    trg_file_name = f"{row.project_name}.{row_idx}.{col_idx}.{file_to_hash[value]}.{value[value.rfind('.') + 1 :].lower()}"
                    if value.count(":") == 1:
                        tmp = value.split(":")
                        archiv_file_path = tmp[0]
                        file_path = tmp[1]
                        if archiv_file_path.lower().endswith((".zip", ".eln")):
                            print(
                                f"zip >>>> {archiv_file_path}, {file_path} >>>> {trg_file_name}"
                            )
                            # get_file_from_zip(
                            #     archiv_file_path,
                            #     file_path,
                            #     trg_directory,
                            #     trg_file_name,
                            # )
                        elif archiv_file_path.lower().endswith(
                            (".tar", ".tar.gz", ".tar.bz2", ".tar.xz")
                        ):
                            print(
                                f"tar >>>> {archiv_file_path}, {file_path} >>>> {trg_file_name}"
                            )
                            # get_file_from_tar(
                            #     archiv_file_path,
                            #     file_path,
                            #     trg_directory,
                            #     trg_file_name,
                            # )
                        elif archiv_file_path.lower().endswith((".rar")):
                            print(
                                f"rar >>>> {archiv_file_path}, {file_path} >>>> {trg_file_name}"
                            )
                            # get_file_from_rar(
                            #     archiv_file_path,
                            #     file_path,
                            #     trg_directory,
                            #     trg_file_name,
                            # )
                        elif archiv_file_path.lower().endswith((".7z")):
                            print(
                                f"sevenz >>>> {archiv_file_path}, {file_path} >>>> {trg_file_name}"
                            )
                            # get_file_from_sevenz(
                            #     archiv_file_path,
                            #     file_path,
                            #     trg_directory,
                            #     trg_file_name,
                            # )
                        del tmp, archiv_file_path, file_path
                    else:
                        print(
                            f"file >>>> {src_directory}{os.sep}data{os.sep}{row.project_name}{os.sep}{value} >>>> {trg_file_name}"
                        )
                        # shutil.copy2(
                        #     f"{src_directory}{os.sep}{data}{os.sep}{row.project_name}{value}",
                        #     f"{trg_directory}{os.sep}{trg_file_name}",
                        # )
                    n_entries += 1
                    del trg_file_name
            # DOCUMENT UNPACKING IN YAML
        # print(f"Row {row_idx}, Column {col_idx} = {value}")

        del df_workflow
        cnt[row.parse] += n_entries
print(cnt)